<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/prompts/prompt_mixin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 访问/定制高级模块中的提示

LlamaIndex包含各种高级模块（查询引擎、响应合成器、检索器等），其中许多模块都会进行LLM调用并使用提示模板。

本指南展示了如何通过`get_prompts`来访问任何模块（包括嵌套模块）的提示集，以及如何使用`update_prompts`轻松更新这些提示。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index

In [ ]:
import os
import openai

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)
from IPython.display import Markdown, display

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


## 设置：加载数据，构建索引并获取查询引擎

在这里，我们将在一个玩具数据集（PG的文章）上构建一个向量索引，并访问查询引擎。

查询引擎是一个简单的RAG流水线，包括top-k检索和LLM合成。


下载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
# 加载文档documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# 将日志记录级别设置为DEBUG，以获得更详细的输出query_engine = index.as_query_engine(response_mode="tree_summarize")

In [ ]:
# 定义提示查看函数def display_prompt_dict(prompts_dict):    for k, p in prompts_dict.items():        text_md = f"**提示键**：{k}<br>" f"**文本：**<br>"        display(Markdown(text_md))        print(p.get_template())        display(Markdown("<br><br>"))

## 访问提示

在这里，我们从查询引擎中获取提示。请注意，*所有*提示都会被返回，包括在查询引擎的子模块中使用的提示。这样可以让您集中查看这些提示！


In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:summary_template<br>**Text:** <br>

Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

#### 在响应合成器上检查 `get_prompts`

您还可以在底层的响应合成器上调用 `get_prompts`，您将看到相同的列表。


In [ ]:
prompts_dict = query_engine.response_synthesizer.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: summary_template<br>**Text:** <br>

Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

#### 使用不同的响应合成策略检查`get_prompts`

在这里，我们尝试使用默认的`compact`方法。

我们会发现使用的模板集是不同的；一个是问答模板，另一个是改进模板。


In [ ]:
# 将日志级别设置为DEBUG，以获得更详细的输出query_engine = index.as_query_engine(response_mode="compact")

In [ ]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

#### 放入查询引擎，获取响应


In [ ]:
response = query_engine.query("What did the author do growing up?")
print(str(response))

The author worked on writing and programming outside of school before college. They wrote short stories and tried writing programs on an IBM 1401 computer using an early version of Fortran. They later got a microcomputer and started programming on it, writing simple games and a word processor. They also mentioned their interest in philosophy and AI.


## 自定义提示符

您还可以使用`update_prompts`函数更新/自定义提示符。将参数值传递给键等于提示字典中看到的键的键。

在这里，我们将更改摘要提示，以使用莎士比亚。


In [ ]:
from llama_index.core import PromptTemplate# 重置query_engine = index.as_query_engine(response_mode="tree_summarize")# 莎士比亚！new_summary_tmpl_str = (    "下面是上下文信息。\n"    "---------------------\n"    "{context_str}\n"    "---------------------\n"    "根据上下文信息和非先验知识，以莎士比亚戏剧的风格回答查询。\n"    "查询：{query_str}\n"    "答案：")new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

In [ ]:
query_engine.update_prompts(
    {"response_synthesizer:summary_template": new_summary_tmpl}
)

In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:summary_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query in the style of a Shakespeare play.
Query: {query_str}
Answer: 


<br><br>

In [ ]:
response = query_engine.query("What did the author do growing up?")
print(str(response))

## 从其他模块访问提示

在这里，我们将看一些其他模块：查询引擎、路由器/选择器、评估器等。


In [ ]:
from llama_index.core.query_engine import (
    RouterQueryEngine,
    FLAREInstructQueryEngine,
)
from llama_index.core.selectors import LLMMultiSelector
from llama_index.core.evaluation import FaithfulnessEvaluator, DatasetGenerator
from llama_index.core.postprocessor import LLMRerank

#### 分析提示：路由器查询引擎


In [ ]:
# 设置样本路由查询引擎from llama_index.core.tools import QueryEngineToolquery_tool = QueryEngineTool.from_defaults(    query_engine=query_engine, description="测试描述")router_query_engine = RouterQueryEngine.from_defaults([query_tool])

In [ ]:
prompts_dict = router_query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: summarizer:summary_template<br>**Text:** <br>

Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

#### 分析提示：FLARE查询引擎


In [ ]:
flare_query_engine = FLAREInstructQueryEngine(query_engine)

In [ ]:
prompts_dict = flare_query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: instruct_prompt<br>**Text:** <br>

Skill 1. Use the Search API to look up relevant information by writing     "[Search(query)]" where "query" is the search query you want to look up.     For example:

Query: But what are the risks during production of nanomaterials?
Answer: [Search(What are some nanomaterial production risks?)]

Query: The colors on the flag of Ghana have the following meanings.
Answer: Red is for [Search(What is the meaning of Ghana's flag being red?)],     green for forests, and gold for mineral wealth.

Query: What did the author do during his time in college?
Answer: The author took classes in [Search(What classes did the author take in     college?)].



Skill 2. Solve more complex generation tasks by thinking step by step. For example:

Query: Give a summary of the author's life and career.
Answer: The author was born in 1990. Growing up, he [Search(What did the     author do during his childhood?)].

Query: Can you write a summary of the Great Gatsby.
Answer: The Great Gatsby is a novel written b

<br><br>

**Prompt Key**: query_engine:response_synthesizer:summary_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query in the style of a Shakespeare play.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: lookahead_answer_inserter:answer_insert_prompt<br>**Text:** <br>


An existing 'lookahead response' is given below. The lookahead response
contains `[Search(query)]` tags. Some queries have been executed and the
response retrieved. The queries and answers are also given below.
Also the previous response (the response before the lookahead response)
is given below.
Given the lookahead template, previous response, and also queries and answers,
please 'fill in' the lookahead template with the appropriate answers.

NOTE: Please make sure that the final response grammatically follows
the previous response + lookahead template. For example, if the previous
response is "New York City has a population of " and the lookahead
template is "[Search(What is the population of New York City?)]", then
the final response should be "8.4 million".

NOTE: the lookahead template may not be a complete sentence and may
contain trailing/leading commas, etc. Please preserve the original
formatting of the lookahead template if possible.

NOTE:

NOTE: the exception to the above

<br><br>

#### 分析提示：LLMMultiSelector


In [ ]:
from llama_index.core.selectors import LLMSingleSelector

selector = LLMSingleSelector.from_defaults()

In [ ]:
prompts_dict = selector.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: prompt<br>**Text:** <br>

Some choices are given below. It is provided in a numbered list (1 to {num_choices}), where each item in the list corresponds to a summary.
---------------------
{context_list}
---------------------
Using only the choices above and not prior knowledge, return the choice that is most relevant to the question: '{query_str}'



<br><br>

#### 分析提示：FaithfulnessEvaluator


In [ ]:
evaluator = FaithfulnessEvaluator()

In [ ]:
prompts_dict = evaluator.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: eval_template<br>**Text:** <br>

Please tell if a given piece of information is supported by the context.
You need to answer with either YES or NO.
Answer YES if any of the context supports the information, even if most of the context is unrelated. Some examples are provided below. 

Information: Apple pie is generally double-crusted.
Context: An apple pie is a fruit pie in which the principal filling ingredient is apples. 
Apple pie is often served with whipped cream, ice cream ('apple pie à la mode'), custard or cheddar cheese.
It is generally double-crusted, with pastry both above and below the filling; the upper crust may be solid or latticed (woven of crosswise strips).
Answer: YES
Information: Apple pies tastes bad.
Context: An apple pie is a fruit pie in which the principal filling ingredient is apples. 
Apple pie is often served with whipped cream, ice cream ('apple pie à la mode'), custard or cheddar cheese.
It is generally double-crusted, with pastry both above and below the filling; the upper crust may be s

<br><br>

**Prompt Key**: refine_template<br>**Text:** <br>

We want to understand if the following information is present in the context information: {query_str}
We have provided an existing YES/NO answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
If the existing answer was already YES, still answer YES. If the information is present in the new context, answer YES. Otherwise answer NO.



<br><br>

#### 分析提示：数据集生成器


In [ ]:
dataset_generator = DatasetGenerator.from_documents(documents)

In [ ]:
prompts_dict = dataset_generator.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: text_question_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge.
generate only questions based on the below query.
{query_str}



<br><br>

**Prompt Key**: text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

#### 分析提示：LLMRerank


In [ ]:
llm_rerank = LLMRerank()

In [ ]:
prompts_dict = dataset_generator.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: text_question_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge.
generate only questions based on the below query.
{query_str}



<br><br>

**Prompt Key**: text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>